In [21]:
import pandas as pd
import seaborn as sns
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import gc
import numpy as np
import time
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import numpy as np
import lightgbm as lgb
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.feature_selection import RFECV
import warnings
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
warnings.filterwarnings("ignore")

#from Tuning_parameter import Lgb_params


In [2]:
train = pd.read_csv('training_set_prep.csv')
test = pd.read_csv('testing_set_prep.csv')
gc.collect()

7

In [3]:
def Reduce_mem(df):
    start_mem_usg = df.memory_usage().sum() / (1024*1024)
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    for col in df.keys():
        if df[col].dtype == int:
            Max = df[col].max()
            Min = df[col].min()
            if -128 < Min and Max < 127:
                df[col] = df[col].astype(np.int8)
            elif -32768 < Min and Max < 32767:
                df[col] = df[col].astype(np.int16)
            elif -2147483648 < Min and Max < 2147483647:
                df[col] = df[col].astype(np.int32)
            else:
                df[col] = df[col].astype(np.int)
        elif df[col].dtype == float:
            df[col] = df[col].astype(np.float32)
        else:
            continue
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return df

def Process_fre(df, cols):
    print('Initial Process_fre.....')
    for col in cols:
        vc = df[col].value_counts(dropna=True, normalize=True).to_dict()
        vc[-1] = -1
        new_col = col+'_freq'
        df[col] = df[col].map(vc)
        df[col] = df[col].astype('float32')

    return df

def Separate_dt(df1):   
    df1['Month'] = 0
    df1.loc[df1['locdt'] < 121, 'Month'] = 4
    df1.loc[df1['locdt'] < 91, 'Month']  = 3
    df1.loc[df1['locdt'] < 61, 'Month']  = 2
    df1.loc[df1['locdt'] < 31, 'Month'] = 1
    
    return df1

def Split_group_kfolds(df):
    Train_X = df.drop(['fraud_ind'], axis=1)
    Train_Y = df['fraud_ind']
    Folds  = GroupKFold(n_splits=3)
    Splited_data = Folds.split(Train_X, Train_Y, groups=Train_X['Month']) 
    return Splited_data

def Cal_f1_score(True_y, Pre_y):
    return f1_score(True_y, Pre_y)

def F1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True

In [4]:
train = Separate_dt(train)
train = Reduce_mem(train)
test = Reduce_mem(test)

Memory usage of properties dataframe is : 673.3983001708984  MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  238.01154708862305  MB
This is  35.344839306576695 % of the initial size
Memory usage of properties dataframe is : 183.37189483642578  MB
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  65.54748821258545  MB
This is  35.74565680910705 % of the initial size


# Validation1

In [5]:
Splited_data = Split_group_kfolds(train)
Excluded_cols = ['bacno','cano','locdt', 'loctm','OOOO_tag','scity','cano_max_locdt','cano_min_locdt',
            'cano_stocn_nunique_locdt','cano_stocn_diff_locdt','cano_stocn_min_diff_locdt', 'txkey',
            'cano_stocn_min_locdt','cano_stocn_max_locdt', 'fraud_ind', 'Month', 'is_test']

All_Features = [Feat for Feat in train.columns if Feat not in Excluded_cols]
print('Total Feature : ', len(All_Features))


Total Feature :  42


In [6]:
Lgb_params = {
        'objective': 'binary',
        'max_depth': -1,
        'learning_rate': 0.05,
        "boosting_type": "gbdt",
        'bagging_freq': 4,
#         'num_boost_round' : 100, 
        }

In [7]:
Cate_feat = ['contp','mchno','acqic','mcc','insfg','stocn','stscd','iterm',
             'ovrlt','flbmk','hcefg','flg_3dsmk','ecfg','etymd',
             'cano_diff','change_signal','cano_diff_change_signal','stocn_scity']

# train[Cate_feat] = train[Cate_feat].astype('category')
# test[Cate_feat] = test[Cate_feat].astype('category')
train = Process_fre(train, Cate_feat)
test  = Process_fre(test, Cate_feat)
for col in ['flbmk', 'flg_3dsmk', '14_variables_count']:
    train[col].fillna(-999, inplace = True)
    test[col].fillna(-999, inplace = True)


Initial Process_fre.....
Initial Process_fre.....


In [8]:
X = train[All_Features +['Month']]
Y = train['fraud_ind']

Oof = np.zeros(len(train))
Preds = np.zeros(len(test))

for fold_n, (train_idx, valid_idx) in enumerate(Splited_data):
    print(f'${fold_n+1} fold')
    X_trn, X_val= X[All_Features].iloc[train_idx], X[All_Features].iloc[valid_idx]
    Y_trn, Y_val = Y.iloc[train_idx], Y.iloc[valid_idx]
    print('Hold out ', X.iloc[valid_idx]['Month'].iloc[0], 'month')
    #X_trn, Y_trn = self.Under_sample(X_trn, Y_trn)
    


    trn_data = lgb.Dataset(X_trn, label=Y_trn)
    val_data = lgb.Dataset(X_val, label=Y_val)
    
    clf = lgb.train(params= Lgb_params, train_set= trn_data, 
                                    valid_sets= [trn_data, val_data],
                                    num_boost_round = 300, 
                                    verbose_eval = 20, 
                                    early_stopping_rounds = 50, 
                                    categorical_feature = Cate_feat, 
                                    feval=F1_score, 
                                    evals_result={})
    
    Y_pred_val = clf.predict(X_val)
    Oof[valid_idx] = Y_pred_val
    Preds += clf.predict(test[All_Features]) / 3
    
Oof = np.round(Oof)
Preds = np.round(Preds)
Score = Cal_f1_score(Y, Oof)
print(f' F1 = {Cal_f1_score(Y, Oof)}')


$1 fold
Hold out  1 month
Training until validation scores don't improve for 50 rounds
[20]	training's binary_logloss: 0.0339617	training's f1: 0.404617	valid_1's binary_logloss: 0.0327042	valid_1's f1: 0.421497
[40]	training's binary_logloss: 0.0280788	training's f1: 0.547163	valid_1's binary_logloss: 0.0270394	valid_1's f1: 0.518628
[60]	training's binary_logloss: 0.0253716	training's f1: 0.601903	valid_1's binary_logloss: 0.0249318	valid_1's f1: 0.554385
[80]	training's binary_logloss: 0.0237126	training's f1: 0.634247	valid_1's binary_logloss: 0.0238089	valid_1's f1: 0.574836
[100]	training's binary_logloss: 0.0225746	training's f1: 0.65831	valid_1's binary_logloss: 0.0233004	valid_1's f1: 0.587832
[120]	training's binary_logloss: 0.0216979	training's f1: 0.668941	valid_1's binary_logloss: 0.023489	valid_1's f1: 0.590605
[140]	training's binary_logloss: 0.0212896	training's f1: 0.673023	valid_1's binary_logloss: 0.0237984	valid_1's f1: 0.590673
[160]	training's binary_logloss: 0.02

# Adversarial Validation

In [9]:
def Label_encoding(df, feat_list):
    print('Initial Label_encoding.....')
    #insfg、ecfg、ovrlt、flbmk、flg_3dsmk
    le = LabelEncoder()
    for feat in feat_list:
        df[feat] = le.fit_transform(df[feat].astype(str))
        
    return df

In [10]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
rm = gc.collect()
del rm
train['is_test'] = 0 
test['is_test'] = 1 
traub = Label_encoding(train, ['ecfg', 'flbmk', 'flg_3dsmk', 'insfg', 'ovrlt'])
test = Label_encoding(test, ['ecfg', 'flbmk', 'flg_3dsmk', 'insfg', 'ovrlt'])


All = pd.concat([train, test], axis=0)
All = All
from sklearn.model_selection import StratifiedKFold

Initial Label_encoding.....
Initial Label_encoding.....


In [30]:
len(train) / len(test)

3.6089952924715116

In [34]:
Excluded_cols = ['bacno','cano','locdt', 'loctm','OOOO_tag','scity','cano_max_locdt','cano_min_locdt',
            'cano_stocn_nunique_locdt','cano_stocn_diff_locdt','cano_stocn_min_diff_locdt', 'txkey',
            'cano_stocn_min_locdt','cano_stocn_max_locdt', 'fraud_ind', 'Month', 'is_test']
All_Features = [Feat for Feat in train.columns if Feat not in Excluded_cols]

kfs = {'Kfold' :  KFold(n_splits = 5, shuffle = True, random_state = 42) , 
       'Skfold' : StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)}

eval_preds = np.zeros(len(All))
probs = np.zeros(len(All))

for name, mehtod in kfs.items():
    print(name)
    for fold, (train_index, test_index) in enumerate(mehtod.split(All, All['is_test'])):
        x0, x1 = All[All_Features].iloc[train_index], All[All_Features].iloc[test_index]
        y0, y1 = All['is_test'].iloc[train_index], All['is_test'].iloc[test_index]        
        print(f'Fold {fold+1} count : ', x0.shape[0])
    
        xgb_params = {
            'learning_rate': 0.05, 'max_depth': 4,'subsample': 0.9,
            'objective': 'binary:logistic', 'eval_metric' : 'auc',
            'n_estimators':100, 'scale_pos_weight' : 3.6
            }   
        clf = xgb.XGBClassifier(**xgb_params, seed = 10, verbosity=1)  
    
        
        clf.fit(x0, y0, eval_set=[(x1, y1)],
               eval_metric='logloss', verbose=False,early_stopping_rounds=10)
                
        prval = clf.predict(x1)
        probs[test_index] = clf.predict_proba(x1)[:,1]
        print(roc_auc_score(y1,prval))
        eval_preds[test_index] = prval
    print('Final score : ', roc_auc_score(All['is_test'], eval_preds))

Kfold
Fold 1 count :  1554761
0.5262733828090991
Fold 2 count :  1554761
0.5382353466338712
Fold 3 count :  1554762
0.5392184374773679
Fold 4 count :  1554762
0.539290165322523
Fold 5 count :  1554762
0.5384123630993248
Final score :  0.5362588578281333
Skfold
Fold 1 count :  1554761
0.5377905235329946
Fold 2 count :  1554761
0.5270553655390395
Fold 3 count :  1554762
0.5381526093156817
Fold 4 count :  1554762
0.5370804174485158
Fold 5 count :  1554762
0.5255982499995886
Final score :  0.5331354233245313


# Feature Selection

In [36]:
SEED = 42

# train      = train
# Test_df       = test
N_folds       = 3
Target        = 'fraud_ind'


def RFE(clf):
    rfecv = RFECV(estimator=clf, step=1, cv = Split_group_kfolds(train), verbose=2, n_jobs = -1, scoring = 'f1')
    rfecv.fit(train[All_Features], train[Target])
    sel_features = [f for f, s in zip(All_Features, rfecv.support_) if s]
    print('\n The selected features are {}:'.format(sel_features))
    print(len(sel_features))
    
    plt.figure(figsize=(12, 9))
    plt.xlabel('Number of features ')
    plt.ylabel('Cross-validation score (F1_score)')
    plt.plot(range(1, len(rfecv.grid_scores_) + 1) , rfecv.grid_scores_)
    plt.show()

In [37]:
Lgb = lgb.LGBMClassifier( 
            objective = 'binary',
            max_depth =  -1,
            learning_rate = 0.05,
            boosting_type = "gbdt",
            bagging_freq = 4,
#         'num_boost_round' : 100, 
        categorical_feature = Cate_feat
                )

RFE(Lgb)

KeyError: 'Month'

# Tuning parameter

In [ ]:
#Gridsearch
param_grid = {
    'learning_rate': [0.05, 0.025, 0.001],
    
#     'n_estimators': [20, 40]
    
}
clf = lgb.train(params= Lgb_params, train_set= trn_data, 
                valid_sets= [trn_data, val_data],
                num_boost_round = 5, 
                verbose_eval = 100, 
                early_stopping_rounds = 50, 
                categorical_feature = Cate_feat, 
                feval=F1_score, 
                evals_result={})

gbm = GridSearchCV(estimator, param_grid, cv=3)
gbm.fit(X_train, y_train)